In [1]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOllama
import pandas as pd
import json

## 提取出文本中的物品

In [2]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOllama(temperature=0, model="qwen:32b-chat-v1.5-q5_K_M")

text_input = """
林克推开门进入卧室，房间内布置得既实用又典雅。一张宽敞的床位于房间的中心位置，床头靠北墙，床尾朝南，正对着一扇大窗户，这样林克在夜晚可以仰望星空，清晨则能迎接温暖的阳光。
床的左侧，也就是房间的西边，放置着一个带有多个抽屉的书桌，书桌面向东，紧邻窗户，确保了充足的自然光线，书桌上方悬挂着一盏设计简洁的吊灯，为夜间阅读提供了便利。在房间的东南角，
一个高挑的衣柜紧贴墙壁，与床相隔约3米，衣柜的双开门设计方便林克挑选衣物，且内部空间宽敞，足以容纳他的冒险装备。在床的对面，也就是房间的南墙边，放置着一把舒适的靠背椅，椅子旁边是一张小巧的茶几，
两者与床相对，距离床尾约2米，为林克提供了一个放松身心的好地方。整个房间的布局考虑到了空间的流动性和使用的便捷性，每件家具的摆放都恰到好处，既满足了功能需求，又不失为一个温馨的休息之所。
"""

get_object_chain = ChatPromptTemplate.from_template("提取出如下文本中的陈设、家具和物品，输出一个数组，尽量详尽。文本：{text_input}") | chat_model | StrOutputParser()

objects_in_text = get_object_chain.invoke({"text_input": text_input})

print(objects_in_text)

['床', '书桌', '吊灯', '衣柜', '靠背椅', '茶几']


## 在素材库中找到匹配的物品

In [7]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品"),
    ResponseSchema(name="object_in_gallery", description="这是你觉得与用户输入最匹配的物品"),
    ResponseSchema(name="objects_url_in_gallery", description="这是这个最匹配的物品的URL"),
    ResponseSchema(name="objects_x_size", description="这是这个最匹配的物品的X轴尺寸"),
    ResponseSchema(name="objects_y_size", description="这是这个最匹配的物品的Y轴尺寸"),        
    ResponseSchema(name="objects_z_size", description="这是这个最匹配的物品的Z轴尺寸"),           
    ResponseSchema(name="match_score", description="你认为用户输入的匹配与你匹配的接近程度的0-100分")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品:
{objects_in_text}

已入库物品:
{objects_in_gallery}

已入库物品URL:
{objects_url_in_gallery}

已入库物品X轴尺寸:
{objects_x_size}

已入库物品Y轴尺寸:
{objects_y_size}

已入库物品Z轴尺寸:
{objects_z_size}


你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "objects_in_gallery"],
    partial_variables={"format_instructions": format_instructions}
)

# Get your standardized names. You can swap this out with whatever list you want!
df = pd.read_csv('../data/ObjectsInGallery.csv')
objects_in_gallery = ", ".join(df['物品'].values)
objects_url_in_gallery = ", ".join(df['URL'].values)
objects_x_size = df['X轴尺寸'].values
objects_y_size = df['Y轴尺寸'].values
objects_z_size = df['Z轴尺寸'].values

# Your user input
_input = prompt.format_prompt(objects_in_text=objects_in_text, objects_in_gallery=objects_in_gallery, objects_url_in_gallery=objects_url_in_gallery, 
                              objects_x_size=objects_x_size, objects_y_size=objects_y_size, objects_z_size=objects_z_size)

print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

# 提取JSON内容
if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsMatched.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"objects_x_size": string  // 这是这个最匹配的物品的X轴尺寸
	"objects_y_size": string  // 这是这个最匹配的物品的Y轴尺寸
	"objects_z_size": string  // 这是这个最匹配的物品的Z轴尺寸
	"match_score": string  // 你认为用户输入的匹配与你匹配的接近程度的0-100分
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得用户提供的一系列物品名称。
在已入库物品列表中找到最合适的匹配项。
最接近的匹配将是具有最接近语义的匹配。不仅仅是字符串相似。
已入库物品的URL和已入库物品是一一对应关系。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品
	"object_in_gallery": string  // 这是你觉得与用户输入最匹配的物品
	"objects_url_in_gallery": string  // 这是这个最匹配的物品的URL
	"objects_x_size":

,object_in_text,object_in_gallery,objects_url_in_gallery,objects_x_size,objects_y_size,objects_z_size,match_score
0,床,床1,D:/Home/Downloads/Furniture/Bed_01/Bed_01.usd,200,100,150,100
1,书桌,书桌1,D:/Home/Downloads/Furniture/Desk_001/Desk_001.usd,120,80,80,100
2,吊灯,无匹配项,,,,,0
3,衣柜,衣柜1,D:/Home/Downloads/Furniture/Armoire_001/Armoir...,120,200,50,100
4,靠背椅,靠背椅1,D:/Home/Downloads/Furniture/Chair_001/Chair_00...,60,120,60,100
5,茶几,无匹配项,,,,,0


## 根据文本确定物品方位和详细的相对位置

In [26]:
# 使用self-discover方法
from langchain import hub

select_prompt = hub.pull("superfhwl/self-discovery-select")
adapt_prompt = hub.pull("superfhwl/self-discovery-adapt")
structured_prompt = hub.pull("superfhwl/self-discovery-structure")
reasoning_prompt = hub.pull("superfhwl/self-discovery-reasoning")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

select_chain = select_prompt | chat_model | StrOutputParser()
adapt_chain = adapt_prompt | chat_model | StrOutputParser()
structure_chain = structured_prompt | chat_model | StrOutputParser()
reasoning_chain = reasoning_prompt | chat_model | StrOutputParser()
overall_chain = (
    RunnablePassthrough.assign(selected_modules=select_chain)
    .assign(adapted_modules=adapt_chain)
    .assign(reasoning_structure=structure_chain)
    .assign(answer=reasoning_chain)
)

with open("../data/ReasoningModules.json", 'r') as f:
    reasoning_modules = json.load(f)

reasoning_modules_str = "\n".join(reasoning_modules)


# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

# 根据匹配信息整理输入布局的数据
objects_matched = []
for obj in ObjectsMatched:
    if int(obj['match_score']) > 0:
        obj_mat = {'物品名称':obj['object_in_text'], 
                    'X轴尺寸':obj['objects_x_size'], 
                    'Y轴尺寸':obj['objects_y_size'], 
                    'Z轴尺寸':obj['objects_z_size']}
        objects_matched.append(obj_mat)


task = f"""
您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户输入的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800

用户输入的物品信息:
{objects_matched}

用户提供的一段文本:
{text_input}

布局方案的格式：
物品1：方位、朝向、与物品2（如果相邻的话）的相对位置、与物品3（如果相邻的话）的相对位置
"""

output = overall_chain.invoke(
    {"task_description": task, "reasoning_modules": reasoning_modules_str}
)

print(output['answer'])


```json
{
   "床": {
     "方位": "中心",
     "朝向": "南",
     "与书桌相对位置": "(X=180, Y=0, Z=250)",
     "与衣柜相对位置": "(X=300, Y=0, Z=250)"
   },
   "书桌": {
     "方位": "紧邻窗户",
     "朝向": "东",
     "与床相对位置": "(X=180, Y=0, Z=125)",
     "与衣柜相对位置": "(X=400, Y=0, Z=125)"
   },
   "衣柜": {
     "方位": "东南角",
     "朝向": "南",
     "与床相对位置": "(X=-300, Y=0, Z=250)",
     "与书桌相对位置": "(X=400, Y=0, Z=125)"
   },
   "靠背椅和茶几": {
     "方位": "床对面南墙边",
     "朝向": "南",
     "与床相对位置": "(X=0, Y=-250, Z=125)",
     "与衣柜相对位置": "(X=400, Y=0, Z=125)"
   }
}
```


In [27]:
for k in output:
    print(k)
    print('==============')
    print(output[k])



task_description

您将获得用户提供的一段文本，描述了房间中的物品和陈设。同时还有一组用户输入的物品名称，它们与文本中的物品和陈设对应。同时还有物品的 X、Y、Z轴尺寸，以厘米为单位。
分析并输出一个物品和陈设的布局方案，符合文本描述的内容。同时要符合人们日常生活的习惯，考虑生活便利性。物品和物品之间不要出现重叠和碰撞。
X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800

用户输入的物品信息:
[{'物品名称': '床', 'X轴尺寸': '200', 'Y轴尺寸': '100', 'Z轴尺寸': '150'}, {'物品名称': '书桌', 'X轴尺寸': '120', 'Y轴尺寸': '80', 'Z轴尺寸': '80'}, {'物品名称': '衣柜', 'X轴尺寸': '120', 'Y轴尺寸': '200', 'Z轴尺寸': '50'}, {'物品名称': '靠背椅', 'X轴尺寸': '60', 'Y轴尺寸': '120', 'Z轴尺寸': '60'}]

用户提供的一段文本:

林克推开门进入卧室，房间内布置得既实用又典雅。一张宽敞的床位于房间的中心位置，床头靠北墙，床尾朝南，正对着一扇大窗户，这样林克在夜晚可以仰望星空，清晨则能迎接温暖的阳光。
床的左侧，也就是房间的西边，放置着一个带有多个抽屉的书桌，书桌面向东，紧邻窗户，确保了充足的自然光线，书桌上方悬挂着一盏设计简洁的吊灯，为夜间阅读提供了便利。在房间的东南角，
一个高挑的衣柜紧贴墙壁，与床相隔约3米，衣柜的双开门设计方便林克挑选衣物，且内部空间宽敞，足以容纳他的冒险装备。在床的对面，也就是房间的南墙边，放置着一把舒适的靠背椅，椅子旁边是一张小巧的茶几，
两者与床相对，距离床尾约2米，为林克提供了一个放松身心的好地方。整个房间的布局考虑到了空间的流动性和使用的便捷性，每件家具的摆放都恰到好处，既满足了功能需求，又不失为一个温馨的休息之所。


布局方案的格式：
物品1：方位、朝向、与物品2（如果相邻的话）的相对位置、与物品3（如果相邻的话）的相对位置

reasoning_modules
1. 我怎样才能设计一个实验来解决这个问题呢?2. 把解决这个问题的想法列一个清单，然后一个接一个地应用到

## 根据方位和相对位置，转换为坐标

In [28]:

response_schemas = [
    ResponseSchema(name="object_in_text", description="这是用户输入的物品名称"),
    ResponseSchema(name="object_name", description="这是用户输入的物品的英语翻译（合法的C语言变量名称形式）"),
    ResponseSchema(name="translation_x", description="这是物品的位置X轴坐标，单位是厘米"),
    ResponseSchema(name="translation_y", description="这是物品的位置Y轴坐标，单位是厘米"),
    ResponseSchema(name="translation_z", description="这是物品的位置Z轴坐标，单位是厘米"),
    ResponseSchema(name="rotation_y", description="这是朝向角度（南=0、东=90、北=180、西=270）")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (output_parser.get_format_instructions())

template = """
您将获得一组用户输入的物品名称，以及物品的 X、Y、Z轴尺寸，以厘米为单位。同时还有一段用户输入的文本，描述了物品的位置、朝向、相对关系。
其中：X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800

根据上述信息输出：

{format_instructions}

用左括号和右括号包装你的最终输出(一个json对象列表)

用户输入的物品信息:
{objects_matched}

用户提供的一段文本:
{text_input}

你的反应:
"""

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(template)  
    ],
    input_variables=["object_in_text", "text_input"],
    partial_variables={"format_instructions": format_instructions}
)

# Your user input
_input = prompt.format_prompt(objects_matched=objects_matched, text_input=output['answer'])


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].strip()
else:
    json_string = output.content

if "```" in json_string:
    json_string = json_string.split("```")[0].strip()

print(json_string)

# output_parser.parse(output.content) Ideally this works but not in all cases
structured_data = json.loads(json_string)

# 存储为JSON文件
with open("../data/ObjectsLayout.json", 'w') as f:
    json.dump(structured_data, f, ensure_ascii=False, indent=4)

# 表格展示
pd.DataFrame(structured_data)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"translation_x": string  // 这是物品的位置X轴坐标，单位是厘米
	"translation_y": string  // 这是物品的位置Y轴坐标，单位是厘米
	"translation_z": string  // 这是物品的位置Z轴坐标，单位是厘米
	"rotation_y": string  // 这是朝向角度（南=0、东=90、北=180、西=270）
}
```
There are 1 message(s)
Type: <class 'langchain_core.messages.human.HumanMessage'>
---------------------------

您将获得一组用户输入的物品名称，以及物品的 X、Y、Z轴尺寸，以厘米为单位。同时还有一段用户输入的文本，描述了物品的位置、朝向、相对关系。
其中：X轴指向东、Y轴指向上、Z轴指向南。场景中心坐标为 X=0,Y=0,Z=0。场景中心朝向为南，场景尺寸为X=800,Y=500,Z=800

根据上述信息输出：

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"object_in_text": string  // 这是用户输入的物品名称
	"object_name": string  // 这是用户输入的物品的英语翻译（合法的C语言变量名称形式）
	"translation_x": string  // 这是物品的位置X轴坐标，单

,object_in_text,object_name,translation_x,translation_y,translation_z,rotation_y
0,床,bed,-100,0,250,0
1,书桌,desk,300,0,125,90
2,衣柜,wardrobe,-400,0,125,0
3,靠背椅和茶几,back_chair_and_coatrack,400,250,125,0


## 将JSON文件转换为USDA文件

In [30]:
import json
import os
from pxr import Usd, UsdGeom, Sdf, Gf

# 定义一个函数来创建USD对象
def create_usd_object(stage, prim_path, obj_name, obj_url, position, rotation):
    # 创建一个新的Prim在给定的路径上
    prim = stage.DefinePrim(prim_path, 'Xform')
    prim.name = obj_name

    xformable = UsdGeom.Xformable(prim)

    xformable.AddTranslateOp().Set(value=translation)
    xformable.AddRotateXYZOp().Set(value=rotation)

    # 创建payload，并设置其references指向目标USD文件
    payload = Sdf.Payload(obj_url)
    prim.GetPayloads().AddPayload(payload)

    print(xformable)
    
    return prim


# 创建USD stage
stage = Usd.Stage.CreateNew('../data/link_room_layout_0.usda')

# 创建一个新的Prim，这将作为默认Prim
default_prim = stage.DefinePrim('/Objects')

# 使用SetDefaultPrim方法设置默认Prim
stage.SetDefaultPrim(default_prim)

# 读取JSON数据
with open('../data/ObjectsMatched.json', 'r') as f:
    ObjectsMatched = json.load(f)

with open('../data/ObjectsLayout.json', 'r') as f:
    ObjectsLayout = json.load(f)

# # 合并两个JSON文件的数据
for obj in ObjectsMatched:
    if int(obj['match_score']) <= 0:
        continue

    for obj_layout in ObjectsLayout:
        if obj_layout['object_in_text'] == obj['object_in_text']:
            obj_layout.update(obj)

for o in ObjectsLayout:
    print(o)

for o in ObjectsMatched:
    print(o)

# 遍历JSON数据，为每个对象创建一个USD prim
for obj in ObjectsLayout:
    if not 'objects_url_in_gallery' in obj:
        continue

    prim_path = '/Objects/' + obj['object_name']
    translation = Gf.Vec3f(float(obj['translation_x']), float(obj['translation_y']), float(obj['translation_z']))
    rotation = Gf.Vec3f(0, float(obj['rotation_y']), 0)
    create_usd_object(stage, prim_path, obj['object_in_text'], obj['objects_url_in_gallery'], translation, rotation)

# 保存USD stage
stage.Save()


{'object_in_text': '床', 'object_name': 'bed', 'translation_x': '-100', 'translation_y': '0', 'translation_z': '250', 'rotation_y': '0', 'object_in_gallery': '床1', 'objects_url_in_gallery': 'D:/Home/Downloads/Furniture/Bed_01/Bed_01.usd', 'objects_x_size': '200', 'objects_y_size': '100', 'objects_z_size': '150', 'match_score': '100'}
{'object_in_text': '书桌', 'object_name': 'desk', 'translation_x': '300', 'translation_y': '0', 'translation_z': '125', 'rotation_y': '90', 'object_in_gallery': '书桌1', 'objects_url_in_gallery': 'D:/Home/Downloads/Furniture/Desk_001/Desk_001.usd', 'objects_x_size': '120', 'objects_y_size': '80', 'objects_z_size': '80', 'match_score': '100'}
{'object_in_text': '衣柜', 'object_name': 'wardrobe', 'translation_x': '-400', 'translation_y': '0', 'translation_z': '125', 'rotation_y': '0', 'object_in_gallery': '衣柜1', 'objects_url_in_gallery': 'D:/Home/Downloads/Furniture/Armoire_001/Armoire_001.usd', 'objects_x_size': '120', 'objects_y_size': '200', 'objects_z_size': '5